### Chains in LangChain

#### Outline
- LLM Chain
- Sequential Chains
    - SimpleSequesntialChain
    - SequentialChain
- RouterChain -

Chains - Chains are the most important building block of LangChain. The chain usually combines an LLM with a prompt. We put put these building blocks together to carry out a sequence of operations on our text or on our other data. These chains can be run over many inputs at a time. 


In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [7]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [8]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


### LLM Chain

LLM Chain is a simple but powerful chain that underpins a lot of other chains. LLM chain is the combination of LLM and prompt.

In [9]:
# Import OpenAI model(LLM), ChatPromptTemplate(Prompt) and LLM chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [49]:
# Initialize the language model i.e ChatOpenAI with a high temperature
llm = ChatOpenAI(temperature=0.9)

In [50]:
# Initialize a prompt. This prompt is going to take in a variable called product. 
# It's going to ask the LLM to generate what the best name is to describe a company that makes that product. 
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [51]:
# Combine LLM and Prompt into a chain. This is what we call an LLM chain. 
chain = LLMChain(llm=llm, prompt=prompt)


In [52]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Rest Linens"'

chain.run formats the prompt under the hood and then passes the whole prompt into the LLM. 

### SequentialChain

Sequential chains is another type of chains. It combines multiple chains where the output of one chain is the input of the next chain. It runs a sequence of chains one after another. 

There are 2 type of sequential chains:
1. SimpleSequentialChain - single input/putput
2. SequentialChain - multiple inputs/outputs

### SimpleSequentialChain

SimpleSequentialChain works best when we have sub chains that expects only one inputs and returns only one output

In [53]:
from langchain.chains import SimpleSequentialChain

In [54]:
# This is going to be 1st chain. This uses an LLM and a prompt. 
# This prompt is going to take in the product and will return the best name to describe that company. 

llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [55]:
# This is going to be second chain. This second chain takes the company name as input and outputs a 20-word description of that company.
#  Here output of the first chain i.e the company name is passed into second chain. 

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

We can easily create a simple sequential chain where we will use the two chains described above. 

In [56]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [57]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Royal Rest Beddings"
Royal Rest Beddings sells luxury mattresses and bedding products that provide ultimate comfort and support for a rejuvenating sleep experience.

> Finished chain.


'Royal Rest Beddings sells luxury mattresses and bedding products that provide ultimate comfort and support for a rejuvenating sleep experience.'

We are going to discuss another example of SimpleSequentialChain as below:

In [10]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [11]:
# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=.7)
template = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [12]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [13]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [14]:
review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...
 

Set in a small beach town, the play follows the journey of two best friends, Sam and Jack. After a summer of fun and laughs, Sam and Jack's friendship is tested when they find themselves in a dark situation. At sunset, they stumble upon a group of thugs who are intent on robbing them. Despite their pleas for mercy, the thugs do not relent. In the ensuing struggle, Sam is murdered and Jack is left with the grief of losing his closest friend. As the tragedy unfolds, the story dives into the complexities of friendship, justice, and revenge. The play ultimately serves as a reminder that tragedy can strike even in the most tranquil of settings.


"Set in a small beach town, the play 'Sam and Jack' offers an intimate, yet powerful look into the complexities of friendship, justice, and revenge. Through its gripping narrative, the story follows the journey of two best friends, Sam and Jack, whose summer of fun and laughter is abruptly interrup

In [15]:
print(review)



"Set in a small beach town, the play 'Sam and Jack' offers an intimate, yet powerful look into the complexities of friendship, justice, and revenge. Through its gripping narrative, the story follows the journey of two best friends, Sam and Jack, whose summer of fun and laughter is abruptly interrupted when they find themselves in a dark situation. After stumbling upon a group of thugs intent on robbing them, Sam is tragically murdered and Jack is left with the grief of losing his closest friend. 

The play is an emotional rollercoaster that touches upon the raw emotions of guilt, regret, sadness, and anger. The cast and crew do an exceptional job of taking the audience through this emotional journey, and the play serves as a reminder that tragedy can strike even in the most tranquil of settings. 'Sam and Jack' is a must-see for anyone looking for a poignant and thought-provoking story that will stay with you long after the curtains close."


Simple sequential chain works well when there's only a single input and a single output. We can run this chain over any product description. In case of multiple inputs or multiple outputs, we use SequentialChain. So, we are going to create a bunch of chains that we're going to use one after another.

### SequentialChain

We will create following chains, that we will use one after another.
1. Take a review and translate it into English. 
2. Create a summary of review in one sentence. 
3. Detect the language of review in the first place.
4. Takes multiple inputs. It takes in the summary variable, which we calculated with the second chain, and the language variable, which we calculated with the third chain. It asks for a follow-up response to the summary in the specified language. 

In [58]:
from langchain.chains import SequentialChain

In [59]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )


In [60]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [61]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [62]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [63]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [64]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam does not hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the taste and texture of the product, suspecting that they may have received an old or counterfeit batch compared to the ones they usually purchase from stores.',
 'followup_message': "Réponse de suivi :\n\nCher(e) client(e),\n\nNous avons bien pris en compte votre commentaire concernant notre produit et nous sommes vraiment désolés que vous ayez été déçu(e) par son goût et sa texture. Nous comprenons votre préoccupation quant à la possibilité de recevoir un lot ancien ou contrefait, différent de ceux que vous achetez habituellement en magasin.\n\nNous tenons à vous assurer que la qualité de 

Points to note:One important thing to note about all these subchains 
1. Input keys and output keys need to be pretty precise. We have used review, English review, summary, language and followup_message
2. The simple sequential chain takes in multiple chains, where each one has a single input and a single output.
3. SimpleSequentialChain takes in multiple chains where each one has a single input and a single output. Here, one chain feeds into other chain, one after another.
4. SequentialChain can take in multiple input variables which is useful when you have more complicated downstream chains that need to be a composition of multiple previous chains. We can easily combine all these chains in sequential chain.

### Router Chain

It is used for do something more complicated. For example, a pretty common but basic operation is to route an input to a chain depending on what exactly that input is. 

If we have multiple subchains, each of which specialized for a particular type of input, we could have a router chain which first decides which subchain to pass it to and then passes it to that chain.

For example, we can route between multiple subchains depending each specialized for a particular type of input. We can have a router chain, which first decides which subchain to pass it and then pass to that chain. 

If we are routing between different types of chains based on subjects like Maths, Physics etc. Based on the subject,  we can have different prompts for them. One prompt can answer physics questions. Second prompt can answer math questions, third can answer history and the fourth prompt can answer computer science. 

We can define prompt templates for these subjects. We can provide more information about these prompt templates as well, for example, we can give each template a name and a description. Now, we can pass this information to the router chain and the router chain can decide when to use which subchain. 

In [65]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [66]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [67]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

1. We need a multi-prompt chain, which is a specific type of chain that is used when routing between multiple different prompt templates. 
2. We also need LLMRouterChain, which uses a language model to route between different subchains. We will use description and the name provided by us above. We'll also import a router output parser, which parses the LLM output into a dictionary that can be used downstream to determine which chain to use and what the input to that chain should be. 


In [68]:
# Define the language model that we will use. 
llm = ChatOpenAI(temperature=0)

In [69]:
# Create destination chains
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

Destination chains are the chains which gets called by RouterChain. Each destination chain itself is a language model chain, an LLM chain. We also define a default chain, in addition to the destination chains. This default chain gets called when the router can't decide which of the subchains to use. This might be called when the input question has nothing to do with physics, math, history, or computer science. 

In [70]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

Now we define template that is used by the LLM to route between the different chains. This has instructions of the task to be done, as well as the specific formatting that the output should be in. 

In [71]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

1. Create full router template by formatting it with destinations that we defined above.
2. This template is flexible to different types of destinations. We can add different types of destinations like English or Latin apart from Physics, Math, History or Computer Science defined above.
3. Create the prompt template from this template.
4. Create the router chain by passing in the LLM and overall router prompt. We have used router output parser as well which helps router chain to decide subchains to route between. 
5. Finally create the overall chain, which has a router chain, destination chains and default chain. 

In [72]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [73]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [74]:
# Ask some questions to RouterChain
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an object that absorbs all incident radiation and reflects or transmits none. It is called "black body" because it absorbs all wavelengths of light, appearing black at room temperature. \n\nAccording to Planck\'s law, black body radiation is characterized by a continuous spectrum of wavelengths and intensities, which depend on the temperature of the object. As the temperature increases, the peak intensity of the radiation shifts to shorter wavelengths, resulting in a change in color from red to orange, yellow, white, and eventually blue at very high temperatures.\n\nBlack body radiation is a fundamental concept in physics and has significant applications in various fields, including astrophysics, thermodynamics, and quantum mechanics. It played a crucial role in the development of quantum theory, as Max Planck\'s explanation of black body radiation led to the birth of quantum mechanics.'

When we ask question related to Physics, it gets routed to the physics chain with the input being the question asked. We can also try with other types of subchains defined above as well. Thus, a math question is routed to the Math subchain. In cases, we pass a question that is not related to any of the defined subchains, the subchain choosen is None and this question is passed to a default subchain and thus it makes a generic call to the language model. 

In [75]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'Thank you for your kind words! As a mathematician, I am happy to help with any math questions, no matter how simple or complex they may be.\n\nThe question you\'ve asked is a basic addition problem: "What is 2 + 2?" To solve this, we can simply add the two numbers together:\n\n2 + 2 = 4\n\nTherefore, the answer to the question "What is 2 + 2?" is 4.'

In [76]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information necessary for the synthesis of proteins, which are essential for the structure and function of cells. It serves as a blueprint for the production of specific proteins that determine the characteristics and traits of an organism. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next, ensuring the continuity of life. Therefore, every cell in our body contains DNA to ensure proper cellular function and to pass on genetic information to future generations.'